In [ ]:
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=qZ4GxOw4U5_AZCc5ab5TH58UCxaD7NFuprTy7ReXyTU&tc=W6cgbYRzZbOPWLJc9gU9_VGZp9u8yG0rVdGgX1SHL3M&cc=6l-oYL_MgrsK-OaNiHNouTZO4T2mZPbWZWWd3nRFf8g

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AVHEtk6wEZB6T2_2Gg8GBK34GtGVotF9GJmMuT7vtldMc9V4bxmU7Q5x-aE

Successfully saved authorization token.


In [ ]:
# Define a function to calculate NIRv with quality control
def NIRv(image):
  nirv=image.expression(
      'nir *0.0001 * float((nir*0.0001 - red*0.0001)/(nir*0.0001 + red*0.0001)-L)',
      {
        'nir': image.select('Nadir_Reflectance_Band2'),
        'red': image.select('Nadir_Reflectance_Band1'),
        'L': 0.08
      })
  #qa1 = image.select('BRDF_Albedo_Band_Mandatory_Quality_Band1')
  #qa2 = image.select('BRDF_Albedo_Band_Mandatory_Quality_Band2')
  #mask = qa1.bitwiseAnd(2**0).eq(0) and qa2.bitwiseAnd(2**0).eq(0) # Bit 0 is set to 1 for low-quality pixels
  return image.addBands(nirv.rename('NIRv'))

In [ ]:
# Set up a mask to remove low-quality pixels based on QA band
def c2mask(image):
    qa1=image.select('BRDF_Albedo_Band_Mandatory_Quality_Band1').unmask()
    qa2=image.select('BRDF_Albedo_Band_Mandatory_Quality_Band2').unmask()
    nirv=image.updateMask(qa1.eq(0) and (qa2.eq(0)))
    return nirv.toFloat()

In [ ]:
# Define the general AOI
aoi = ee.Geometry.Polygon(
        [[[-180, -90],
          [-180, 90],
          [180, 90],
          [180, -90],
          [-180, -90]]],
        None,
        False)

In [ ]:
import numpy as np
import time

In [ ]:
# Define the start and end dates for the study period -- LOOP
md=['01-01','01-05','01-06','01-10','01-11','01-15','01-16','01-20','01-21','01-25','01-26','01-30',
    '01-31','02-04','02-05','02-09','02-10','02-14','02-15','02-19','02-20','02-24','02-25','03-01',
    '03-02','03-06','03-07','03-11','03-12','03-16','03-17','03-21','03-22','03-26','03-27','03-31',
    '04-01','04-05','04-06','04-10','04-11','04-15','04-16','04-20','04-21','04-25','04-26','04-30',
    '05-01','05-05','05-06','05-10','05-11','05-15','05-16','05-20','05-21','05-25','05-26','05-30',
    '05-31','06-04','06-05','06-09','06-10','06-14','06-15','06-19','06-20','06-24','06-25','06-29',
    '06-30','07-04','07-05','07-09','07-10','07-14','07-15','07-19','07-20','07-24','07-25','07-29',
    '07-30','08-03','08-04','08-08','08-09','08-13','08-14','08-18','08-19','08-23','08-24','08-28',
    '08-29','09-02','09-03','09-07','09-08','09-12','09-13','09-17','09-18','09-22','09-23','09-27',
    '09-28','10-02','10-03','10-07','10-08','10-12','10-13','10-17','10-18','10-22','10-23','10-27',
    '10-28','11-01','11-02','11-06','11-07','11-11','11-12','11-16','11-17','11-21','11-22','11-26',
    '11-27','12-01','12-02','12-06','12-07','12-11','12-12','12-16','12-17','12-21','12-22','12-26',
    '12-27','12-31']

In [ ]:
#test start & end date -- single year
for yr in range(2002,2003):
  for pd in range(73):
    #print(pd)
    td1=md[pd*2]
    td2=md[pd*2+1]
    start_date = str(yr)+'-'+td1
    end_date   = str(yr)+'-'+td2
    #print(start_date,end_date)
    #Bingo

In [ ]:
#cool & then save the array to google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

In [ ]:
#full readin 2002-2023
for yr in range(2002,2023):
  for pd in range(73):
    print(pd)
    td1=md[pd*2]
    td2=md[pd*2+1]
    start_date = str(yr)+'-'+td1
    end_date   = str(yr)+'-'+td2
    print(start_date,end_date)

    # Load the MCD43A4 data for the study period and filter by the study area
    mcd43a4 = ee.ImageCollection('MODIS/006/MCD43A4') \
            .filterDate(start_date, end_date) \
            .filterBounds(aoi)

    # Map the function over the image collection
    nirv_collection = mcd43a4.map(NIRv)
    # Map the function over the NIRv collection
    masked_nirv_collection = nirv_collection.map(c2mask).mean()
    projection=masked_nirv_collection.select('NIRv').projection()
    transform1 = projection.getInfo()['transform']
    resampled_image = masked_nirv_collection.reproject(crs=masked_nirv_collection.select('NIRv').projection(), crsTransform=transform1)

    #convert to array
    start_time=time.time()
    na_glo=np.ones((130,350))*-1
    roi=ee.Geometry.Rectangle([-170.00, -60.00, 180.00, 70.00],proj='EPSG:4326')
    nirv_array3 = np.array(resampled_image.sampleRectangle(region=roi, defaultValue=-1).getInfo()['properties']['NIRv'])
    na_glo=nirv_array3
    #save to google drive
    folder_name = 'NIRv_array_real_1deg/'+str(yr)
    folder_path = '/content/drive/My Drive/' + folder_name
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    array_file_path = folder_path + '/NIRV_'+str(yr)+'_'+str(pd+1)+'.npy'
    np.save(array_file_path, na_glo)

    seconds=time.time()-start_time
  print('Time taken:',time.strftime("%H:%M:%S",time.gmtime(seconds)))

0
2002-01-01 2002-01-05
1
2002-01-06 2002-01-10
2
2002-01-11 2002-01-15
3
2002-01-16 2002-01-20
4
2002-01-21 2002-01-25
5
2002-01-26 2002-01-30
6
2002-01-31 2002-02-04
7
2002-02-05 2002-02-09
8
2002-02-10 2002-02-14
9
2002-02-15 2002-02-19
10
2002-02-20 2002-02-24
11
2002-02-25 2002-03-01
12
2002-03-02 2002-03-06
13
2002-03-07 2002-03-11
14
2002-03-12 2002-03-16
15
2002-03-17 2002-03-21
16
2002-03-22 2002-03-26
17
2002-03-27 2002-03-31
18
2002-04-01 2002-04-05
19
2002-04-06 2002-04-10
20
2002-04-11 2002-04-15
21
2002-04-16 2002-04-20
22
2002-04-21 2002-04-25
23
2002-04-26 2002-04-30
24
2002-05-01 2002-05-05
25
2002-05-06 2002-05-10
26
2002-05-11 2002-05-15
27
2002-05-16 2002-05-20
28
2002-05-21 2002-05-25
29
2002-05-26 2002-05-30
30
2002-05-31 2002-06-04
31
2002-06-05 2002-06-09
32
2002-06-10 2002-06-14
33
2002-06-15 2002-06-19
34
2002-06-20 2002-06-24
35
2002-06-25 2002-06-29
36
2002-06-30 2002-07-04
37
2002-07-05 2002-07-09
38
2002-07-10 2002-07-14
39
2002-07-15 2002-07-19
40
2002-07